In [20]:
import pandas as pd
from tensorflow.keras.models import load_model
import pickle


In [21]:
# loading model.h5, scaler.pkl, labelencoder.pkl, ohe.pkl

model = load_model('model.h5')

with open('LabelEncoder_gender.pkl', 'rb') as file:
    LabelEncoder_gender = pickle.load(file)

with open('OHE_geography.pkl', 'rb') as file:
    OHE_geography = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [22]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [23]:
new_data = pd.DataFrame([input_data])
new_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [24]:
new_data['Gender'] = LabelEncoder_gender.transform(new_data['Gender'])
new_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [25]:
OHE_geo = OHE_geography.transform(new_data[['Geography']]).toarray()
OHE_geo

array([[1., 0., 0.]])

In [26]:
OHE_Geo_df = pd.DataFrame(OHE_geo, columns= OHE_geography.get_feature_names_out(['Geography']))
OHE_Geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [27]:
pred_data  = pd.concat([new_data.drop('Geography', axis=1),OHE_Geo_df], axis=1)
pred_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [28]:
# For column names and data types
print(pred_data.dtypes)


CreditScore            int64
Gender                 int64
Age                    int64
Tenure                 int64
Balance                int64
NumOfProducts          int64
HasCrCard              int64
IsActiveMember         int64
EstimatedSalary        int64
Geography_France     float64
Geography_Germany    float64
Geography_Spain      float64
dtype: object


In [29]:
scaled = scaler.transform(pred_data)
scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [30]:
pred = model.predict(scaled)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step


array([[0.1987603]], dtype=float32)

In [31]:
predicted_value = pred[0][0]
predicted_value

np.float32(0.1987603)

In [32]:
if predicted_value > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
